In [197]:
import kfp
import yaml
import re

c = kfp.Client()
# Get all the pipelines (100 is max call in api)
tmp_ls_pipelines = c.list_pipelines('',100,'')
ls_pipelines = tmp_ls_pipelines.pipelines
while tmp_ls_pipelines.next_page_token is not None:
    tmp_ls_pipelines = c.list_pipelines(tmp_ls_pipelines.next_page_token,100,'')
    ls_pipelines = ls_pipelines + tmp_ls_pipelines.pipelines

In [198]:
def seq_iter(obj):
    return obj if isinstance(obj, dict) else range(len(obj))

# Recursive function to check against regex
# yaml_file : the part of the yaml still in need of validation
# parent_keys : to keep track of where the hits are
# excluded_keys : to ignore anything under that key
def recursive_secret_check(yaml_file, parent_keys=None, excluded_keys=None):
    positive_tests = []
    if parent_keys is None:
        parent_keys = []
    if excluded_keys is None:
        excluded_keys = ['secretKeyRef']
    if isinstance(yaml_file, str):
        if re.search(r"^secret$|^password$", yaml_file.lower()):
            positive_tests.append(parent_keys)  # Record the flagged key
    else:
        for k in seq_iter(yaml_file):
            if k in excluded_keys:
                continue  # (next step in for loop)
            full_key_context = list(parent_keys)  # make a copy of the parent_keys
            full_key_context.append(k)  # appends a second list of just this key k
            # Skip the check if it's not string
            if isinstance(k, str) and re.search(r"^secret$|^password$", k.lower()):
                positive_tests.append(full_key_context)  # Record the flagged key
            #Recursive call to test deeper
            deeper_positives = recursive_secret_check(yaml_file[k], full_key_context)
            #Concatenate result
            positive_tests = positive_tests + deeper_positives
    return positive_tests

In [199]:
# Loop for each of the pipeline
for p_pipeline in ls_pipelines:
    print(p_pipeline.name)
    #For each pipeline, get the versions
    p_pipeline_versions = c.pipelines.list_pipeline_versions(resource_key_id = p_pipeline.id, resource_key_type = 'PIPELINE')
    #If no version, cannot get templates
    if p_pipeline_versions.total_size is not None:
        for p_version in p_pipeline_versions.versions:
            p_template = c.pipelines.get_pipeline_version_template(p_version.id)
            #each template of each pipeline
            yaml_template = yaml.load(p_template.template, Loader=yaml.BaseLoader)

            res = recursive_secret_check(yaml_template)
            if res:
                print('==================')
                print('Warning: ' + p_pipeline.id + ' version ' + p_version.id)
                print(res)
    else:
        print('Pipeline with no version : ' + p_pipeline.name + ' ' + p_pipeline.id)
print('Check completed')

compute-pi-demo
prov-scraper-kfp
testpipe-447708894
testpipe-448130251
testpipe-448145217
testpipe-449187266
testpipe-449224450
testpipe-449386903
testpipe-449429067
testpipe-449471008
testpipe-449623593
testpipe-449691596
testpipe-449926804
testpipe-449949127
testpipe-449951297
testpipe-450011510
testpipe-450036305
testpipe-451339070
testpipe-451515172
testpipe-451905763
testpipe-454718103
test-lvib7
testing-2020-xqsmf
testing-2020-zbjli
titanic-ml-an6ik
titanic-ml-xqad4
titanic-ml-o34rw
titanic-ml-xfm89
titanic-ml-lchzs
titanic-ml-sud13
titanic-ml-7v9kz
titanic-ml-g9092
titanic-ml-n6mjv
titanic-ml-adjnk
titanic-ml-ldzmo
titanic-ml-4yr5o
titanic-ml-5n3re
Pipeline with no version : titanic-ml-5n3re 205cb39e-7467-46e8-ac97-62262f35dd66
run-ltc-prod-scraper
OpenAddressesPipeline
titanic-ml-sz37x
OpenAddressesPipeline_1CPU
Crop Imaging Events
pipeline-psw-name
[['spec', 'templates', 0, 'inputs', 'parameters', 0, 'name'], ['spec', 'templates', 0, 'inputs', 'parameters', 1, 'name'], ['spec'